In [3]:
import pandas as pd
import json
pd.options.mode.chained_assignment = None

years = ['2008','2009','2010','2011','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021']
for year in years:

    expenses = pd.read_csv('.\\raw_despesas\\Ano-'+year+'.csv', encoding = 'utf8', delimiter=';')
    cpfs = expenses.drop_duplicates(subset='cpf')
    cpfs[['cpf','txNomeParlamentar']].drop_duplicates()
    
    #iterate through deputados
    for depIndex in range(1, len(cpfs)):


        #VIS deputado first node
        depNode = cpfs[['cpf','txNomeParlamentar']].iloc[depIndex] 
        allExpensesSingleDep = expenses[expenses['cpf'] == depNode[0]] #all expenses single dep
        cnpjSingleDep = allExpensesSingleDep.drop_duplicates(subset='txtCNPJCPF')
        cnpjSingleDep[['txtCNPJCPF','txtFornecedor']].drop_duplicates()

        #VIS cnpj single dep nodes
        allCnpjSingleDep = cnpjSingleDep[['txtCNPJCPF','txtFornecedor']] 
        allCnpjSingleDep['txtCNPJCPF']= allCnpjSingleDep['txtCNPJCPF'].str.replace('\.|\/|\-|\ ', '',regex=True)

        #VIS cnpj single dep edges
        allExpensesSingleDep['txtCNPJCPF']= allExpensesSingleDep['txtCNPJCPF'].str.replace('\.|\/|\-|\ ', '',regex=True)
        allExpensesSingleDep = allExpensesSingleDep[['cpf','txtCNPJCPF']]

        #Save Nodes to Json
        depNodeJson = json.loads(depNode.to_json())
        depNodeJson['id'] = depNodeJson.pop('cpf')
        depNodeJson['label'] = depNodeJson.pop('txNomeParlamentar')
        depNodeJson['shape'] = 'circle'

        #set filename as deputado CPF
        filename = str(int(depNodeJson['id']))

        allCnpjSingleDep.rename(columns = {'txtCNPJCPF':'id', 'txtFornecedor':'label'}, inplace = True)
        allCnpjSingleDep.insert(loc=len(allCnpjSingleDep.columns), column='shape', value='box')
        allCnpjSingleDepJson = json.loads(allCnpjSingleDep.to_json(orient="records"))

        #adding deputado node to json array
        allCnpjSingleDepJson.append(depNodeJson.copy())

        out_file = open("./web/despesas/depnodesedges/"+year+"/n"+filename+".json", "w")
        json.dump(allCnpjSingleDepJson,out_file, separators=(',', ':'))
        out_file.close()


        #Save Edges to Json
        allExpensesSingleDep.rename(columns = {'cpf':'from', 'txtCNPJCPF':'to'}, inplace = True)
        allExpensesSingleDepJson = json.loads(allExpensesSingleDep.to_json(orient="records"))

        out_file = open("./web/despesas/depnodesedges/"+year+"/e"+filename+".json", "w")
        json.dump(allExpensesSingleDepJson,out_file, separators=(',', ':'))
        out_file.close() 